In [1]:
from typing import Callable, Any

# from pytube import YouTube
from urllib.parse import urlunsplit, urlencode
from pydantic.dataclasses import dataclass
import dataclasses
import datetime
import time
import os
import json
import boto3
import socket

# import libraries
# from vidgear.gears import CamGear
import cv2
import numpy as np
import fastavro

from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, BinaryType

from fastavro import writer, reader, parse_schema
import io

from classifiers import RawImageRecord

from kafka import kafka_config, raw_video_frames_topic_name, processed_video_frames_topic_name

from classifiers import HAARClassifier, BaseObjectDetector, ProcessedImage


In [3]:

avro_schema = """
        {
            "type": "record",
            "namespace": "com.mycorp.mynamespace",
            "name": "sampleRecord",
            "doc": "Sample schema to help you get started.",
            "fields": [
                {
                    "name": "video_stream_id",
                    "type": "string",
                    "doc": "id for video stream taken from source"
                },
                {
                    "name": "jpeg_image",
                    "type": "bytes",
                    "doc": "jpeg image"
                },
                {
                    "name": "metadata_json",
                    "type": "string",
                    "doc": "Any additional information in json format"
                }
            ]
        }
        """






def bytes_with_schema_to_avro(binary, avro_read_schema=parse_schema(json.loads(avro_schema))):
    with io.BytesIO(binary) as bytes_io:
        reader = fastavro.reader(bytes_io, avro_read_schema)
        return next(reader)

# def main():

    
# .config("spark.jars", "spark-sql-kafka-0-10_2.12-3.5.0.jar")\
# .config("spark.driver.extraClassPath", "spark-sql-kafka-0-10_2.12-3.5.0.jar")\
# .config("spark.executor.extraClassPath", "spark-sql-kafka-0-10_2.12-3.5.0.jar")\
 
# .config("spark.jars", )\

spark = SparkSession.builder\
.config(
    "spark.jars",
    ','.join([
        "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar",
        "https://repo1.maven.org/maven2/org/apache/kafka/kafka-clients/3.5.1/kafka-clients-3.5.1.jar",
        "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10-assembly_2.12/3.5.0/spark-streaming-kafka-0-10-assembly_2.12-3.5.0.jar",
        "https://repo1.maven.org/maven2/org/apache/commons/commons-pool2/2.11.1/commons-pool2-2.11.1.jar",
        "https://repo1.maven.org/maven2/org/apache/spark/spark-token-provider-kafka-0-10_2.12/3.5.0/spark-token-provider-kafka-0-10_2.12-3.5.0.jar"
    ])
)\
.getOrCreate()

stream = (
    spark
    .readStream
    .format("kafka")
    .options(
        **{f"kafka.{k}": v for k, v in kafka_config.items()}
    )
    .option("subscribe", raw_video_frames_topic_name)
    .option("startingOffsets", "earliest")
    .load()
    .select("key", "value")
    .writeStream.format("console").start()
    # .rdd
    # .start()
    # .mapValues(bytes_with_schema_to_avro)
    # .map(lambda key, record: RawImageRecord(key=key, **record))
    # .flatMapValues(lambda record: record.process_image())
    # .map(lambda key, image: {
    #         "key": f"{key}_{image.detector_name}_{image.object_name}",
    #         "value": image.to_json()
    #     }
    # )
)

# import time
# time.sleep(100) # sleep 10 seconds
# stream.stop()

stream.awaitTermination()

# dir(stream)

# stream.show()

# write_schema = StructType([
#     StructField("key", StringType()),
#     StructField("value", StringType())
# ])

# (
#     spark.createDataFrame(stream, schema=write_schema)
#     .writeStream
#     .format('kafka')
#     .options(
#         **{f"kafka.{k}": v for k, v in kafka_config.items()}
#     )
    
#     .option("topic", processed_video_frames_topic_name)
#     # .option("checkpointLocation", "/path/to/checkpoint/dir/in/hdfs/")
#     # .trigger()
#     .save()
# )

# main()

StreamingQueryException: [STREAM_FAILED] Query [id = 7d0e8807-e171-4f45-8def-7701cd57c9a2, runId = bc46a35c-d5e3-47df-925b-d0098e91f5f2] terminated with exception: Failed to create new KafkaAdminClient

In [4]:
kafka_config

{'bootstrap.servers': 'pkc-p11xm.us-east-1.aws.confluent.cloud:9092',
 'security.protocol': 'SASL_SSL',
 'sasl.mechanism': 'PLAIN',
 'sasl.username': 'ZMZ76UXSHPTUA5HO',
 'sasl.password': 'd4j2dCdXsgHyFAY15haq0mBJMoQ+bqPeXlvafabiJtjFjij5VEuy+Dp6r2VD3p3I',
 'client.id': '188ee4ecd2b6'}

In [ ]:
    {
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": bootstrap_servers
    }

In [5]:
{f"kafka.{k}": v for k, v in kafka_config.items()}

{'kafka.bootstrap.servers': 'pkc-p11xm.us-east-1.aws.confluent.cloud:9092',
 'kafka.security.protocol': 'SASL_SSL',
 'kafka.sasl.mechanism': 'PLAIN',
 'kafka.sasl.username': 'ZMZ76UXSHPTUA5HO',
 'kafka.sasl.password': 'd4j2dCdXsgHyFAY15haq0mBJMoQ+bqPeXlvafabiJtjFjij5VEuy+Dp6r2VD3p3I',
 'kafka.client.id': '188ee4ecd2b6'}